- references
    - https://github.com/kuangliu/pytorch-cifar
    - https://github.com/Siddharth1698/ResNet-on-CIFAR-10-dataset.git

In [1]:
import torchvision
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18, resnet34, resnet101, resnet152
from tqdm.notebook import tqdm

In [2]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.CrossEntropyLoss()

In [3]:
def train(model):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    # for batch_idx, (inputs, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    # print(f'train loss: {train_loss/len(train_loader):.3f} | acc: {100.*correct/total:.3f}({correct}/{total})')

In [4]:
def test(model):
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in tqdm(enumerate(test_loader), total=len(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'test loss: {test_loss/len(test_loader):.3f} | acc: {100.*correct/total:.3f}%({correct}/{total})')

## training from scartch

In [58]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [59]:
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=1024, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=250, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [60]:
len(train_set) / train_loader.batch_size

48.828125

In [75]:
# lr = 0.1
# lr = 0.01, 74%
model = resnet18(weights=None, num_classes=len(classes)).to(device) 
# lr = 0.01, 74%
# lr = 0.005, 76% 
# model = resnet34(weights=None, num_classes=len(classes)).to(device) 
# lr = 0.01, 68%
# lr = 0.005, 66%
# model = resnet101(weights=None, num_classes=len(classes)).to(device) 
# lr = 0.01, 
# lr = 0.005, 
# model = resnet152(weights=None, num_classes=len(classes)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

max_epochs = 50
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)
# model.to(device)
for epoch in tqdm(range(max_epochs)):
    train(model)
    scheduler.step()
    if (epoch+1) % 5 == 0:
        test(model)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 1.066 | acc: 63.140%(6314/10000)


  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 0.833 | acc: 73.120%(7312/10000)


...

  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 1.909 | acc: 75.220%(7522/10000)


  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 2.126 | acc: 75.790%(7579/10000)


KeyboardInterrupt: 

## training from pretrained

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=1024, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=250, shuffle=False, num_workers=2)

### Vanilla FC

In [74]:
# lr = 1e-3, 84%
# model = resnet18(weights='DEFAULT')
# lr = 1e-3, 
model = resnet34(weights='DEFAULT')
# model = resnet152(weights='DEFAULT')
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

max_epochs = 50
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)
# model.to(device)
for epoch in tqdm(range(max_epochs)):
    train(model)
    scheduler.step()
    if (epoch+1) % 5 == 0:
        test(model)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/whaow/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 64.0MB/s]


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 0.633 | acc: 82.750%(8275/10000)


  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 0.777 | acc: 82.610%(8261/10000)


  0%|          | 0/40 [00:00<?, ?it/s]

test loss: 0.977 | acc: 83.030%(8303/10000)


KeyboardInterrupt: 

### scale the image to 224

In [5]:
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=.40),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=512, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=250, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# lr = 1e-3, 84%
# model = resnet18(weights='DEFAULT')
# lr = 1e-3, 
model = resnet34(weights='DEFAULT')
# model = resnet152(weights='DEFAULT')
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

max_epochs = 5
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)
# model.to(device)
for epoch in tqdm(range(max_epochs)):
    train(model)
    scheduler.step()
    test(model)

  0%|          | 0/5 [00:00<?, ?it/s]